In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

# Importing Models and Scores:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score

In [76]:
df=pd.read_csv(r'/content/mfcc_40_features_only.csv')
df2=pd.read_csv(r'/content/drive/MyDrive/audios/csv files/Freq-Domain-mean-features.csv')

In [83]:
# 1. Ensure both DataFrames have a common key (e.g., filepath or filename)
# Let's assume the key is 'filepath' in both df and df2

# 2. Select only the relevant columns from df2
selected_cols = ['filepath', 'BER_Mean', 'Spec_Centroid_Mean', 'Spec_Bandwidth_Mean']
df2_selected = df2[selected_cols]

# 3. Merge into df on 'filepath'
df = df.merge(df2_selected, on='filepath', how='left')
df

,filename,emotion,intensity,modality,gender,actor_id,filepath,AE_mean,RMSE_mean,ZCR_mean,...,MFCC_37,MFCC_38,MFCC_39,MFCC_40,BER_Mean_x,Spec_Centroid_Mean_x,Spec_Bandwidth_Mean_x,BER_Mean_y,Spec_Centroid_Mean_y,Spec_Bandwidth_Mean_y
0,03-01-01-01-02-01-22.wav,Neutral,Normal,Audio-Only,Female,22,/content/drive/MyDrive/audios/wav files/Audio_...,0.007719,0.002580,0.059296,...,0.629421,1.542944,0.125471,-0.027604,344.622345,6392.194350,5549.157395,344.622345,6392.194350,5549.157395
1,03-01-01-01-01-02-22.wav,Neutral,Normal,Audio-Only,Female,22,/content/drive/MyDrive/audios/wav files/Audio_...,0.007135,0.002282,0.067220,...,0.757751,0.020876,-0.642699,-0.046494,201.883392,6383.553288,5614.825854,201.883392,6383.553288,5614.825854
2,03-01-01-01-01-01-22.wav,Neutral,Normal,Audio-Only,Female,22,/content/drive/MyDrive/audios/wav files/Audio_...,0.006430,0.002092,0.061312,...,0.610917,0.878997,0.091135,-0.332697,305.614830,6345.728410,5558.905619,305.614830,6345.728410,5558.905619
3,03-01-01-01-02-02-22.wav,Neutral,Normal,Audio-Only,Female,22,/content/drive/MyDrive/audios/wav files/Audio_...,0.007372,0.002385,0.072839,...,0.382474,1.535111,0.098079,0.174156,239.593109,6488.189220,5592.395097,239.593109,6488.189220,5592.395097
4,03-01-03-02-02-01-22.wav,Happy,Strong,Audio-Only,Female,22,/content/drive/MyDrive/audios/wav files/Audio_...,0.039155,0.013946,0.086826,...,1.961565,0.304025,-2.015385,-1.509780,149.006699,6789.160067,5660.422516,149.006699,6789.160067,5660.422516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,03-02-06-02-02-01-02.wav,Fearful,Strong,Audio-Only,Female,2,/content/drive/MyDrive/audios/wav files/Audio_...,0.052375,0.020592,0.086145,...,-5.350123,-0.636383,1.444936,-0.051213,453.503876,4737.180648,4455.844067,453.503876,4737.180648,4455.844067
2448,03-02-06-01-02-01-02.wav,Fearful,Normal,Audio-Only,Female,2,/content/drive/MyDrive/audios/wav files/Audio_...,0.032363,0.012334,0.075225,...,-5.684637,-0.136087,2.389726,1.795649,507.715363,5319.634513,4577.900375,507.715363,5319.634513,4577.900375
2449,03-02-06-01-02-02-02.wav,Fearful,Normal,Audio-Only,Female,2,/content/drive/MyDrive/audios/wav files/Audio_...,0.030873,0.012220,0.081032,...,-3.737191,0.607224,3.287630,2.869979,624.258479,4823.505866,4196.330641,624.258479,4823.505866,4196.330641
2450,03-02-06-02-01-01-02.wav,Fearful,Strong,Audio-Only,Female,2,/content/drive/MyDrive/audios/wav files/Audio_...,0.050430,0.019506,0.078261,...,-5.726693,-1.442012,1.422696,0.313033,406.675842,4909.359382,4300.577625,406.675842,4909.359382,4300.577625


In [84]:
df['intensity'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 2452 entries, 0 to 2451
Series name: intensity
Non-Null Count  Dtype 
--------------  ----- 
2452 non-null   object
dtypes: object(1)
memory usage: 19.3+ KB


In [85]:

# Remove rows where emotion is 'Neutral' or 'Surprised'
df = df[~df['emotion'].isin(['Surprised'])].reset_index(drop=True)

print(f"Original dataset size: {df.shape[0]} rows")
print(f"Filtered dataset size: {df.shape[0]} rows")


Original dataset size: 2260 rows
Filtered dataset size: 2260 rows


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260 entries, 0 to 2259
Columns: 184 entries, filename to Spec_Bandwidth_Mean_y
dtypes: float64(177), int64(1), object(6)
memory usage: 3.2+ MB


## Trying ANN as base model.

In [87]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import joblib

# 1. Drop unused columns
df_model = df.drop(columns=['filename', 'filepath', 'actor_id', 'modality'])

# 2. Label encode categorical columns and save encoders
le_gender = LabelEncoder()
le_intensity = LabelEncoder()
le_emotion = LabelEncoder()

df_model['gender'] = le_gender.fit_transform(df_model['gender'])
df_model['intensity'] = le_intensity.fit_transform(df_model['intensity'])
df_model['emotion'] = le_emotion.fit_transform(df_model['emotion'])

# Save encoders for inference later
joblib.dump(le_gender, "le_genderii.pkl")
joblib.dump(le_intensity, "le_intensityii.pkl")
joblib.dump(le_emotion, "le_emotionii.pkl")

# 3. Split X and y
X = df_model.drop(columns=['emotion', 'AE_mean', 'RMSE_mean', 'ZCR_mean'])
y = df_model['emotion']

# 4. Save feature column order
feature_order = X.columns.tolist()
joblib.dump(feature_order, "feature_order.pkl")

# 5. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 6. Scale only float columns
float_cols = X_train.select_dtypes(include=['float32', 'float64', 'int64']).columns.difference(['gender', 'intensity'])
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[float_cols] = scaler.fit_transform(X_train[float_cols])
X_test_scaled[float_cols] = scaler.transform(X_test[float_cols])

# Save scaler
joblib.dump(scaler, "scalerii.pkl")

# 7. Apply SMOTE
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_scaled, y_train)

print("Training preprocessing complete.")


Training preprocessing complete.


In [88]:
# Show class distribution after SMOTE
import pandas as pd

class_counts = pd.Series(y_train_bal).value_counts()
print("Class distribution after SMOTE:")
print(class_counts.sort_index())


Class distribution after SMOTE:
emotion
0    301
1    301
2    301
3    301
4    301
5    301
6    301
Name: count, dtype: int64


In [89]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report

# Build ANN
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_bal.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(le_emotion.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train_bal, y_train_bal, epochs=250, batch_size=32, validation_split=0.2)

# Evaluate
y_pred_probs = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_probs, axis=1)

print(classification_report(y_test, y_pred, target_names=le_emotion.classes_))

# Save model
model.save("emotion_ann_modelii.h5")


Epoch 1/250
53/53 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.2244 - loss: 1.8952 - val_accuracy: 0.1019 - val_loss: 1.9367
Epoch 2/250
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3006 - loss: 1.7174 - val_accuracy: 0.2085 - val_loss: 1.7421
Epoch 3/250
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3484 - loss: 1.5957 - val_accuracy: 0.1943 - val_loss: 1.6701
Epoch 4/250
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3974 - loss: 1.4933 - val_accuracy: 0.2938 - val_loss: 1.5571
Epoch 5/250
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4633 - loss: 1.3855 - val_accuracy: 0.3697 - val_loss: 1.4329
Epoch 6/250
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4861 - loss: 1.3425 - val_accuracy: 0.4573 - val_loss: 1.4246
Epoch 7/250
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4910 - loss: 1.2734 - val_accuracy: 0.5711 - val_loss: 1.2094
Epoch 8/250
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5001 - loss: 1.2421 - val_accuracy: 0.5877 

              precision    recall  f1-score   support

       Angry       0.95      0.97      0.96        75
        Calm       0.84      0.84      0.84        75
     Disgust       0.81      0.56      0.67        39
     Fearful       0.77      0.83      0.79        75
       Happy       0.83      0.79      0.81        75
     Neutral       0.61      0.82      0.70        38
         Sad       0.81      0.76      0.79        75

    accuracy                           0.81       452
   macro avg       0.80      0.80      0.79       452
weighted avg       0.82      0.81      0.81       452



In [90]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

# y_test: true labels (label encoded)
# y_pred: predicted labels (label encoded)

# If your true labels are one-hot encoded, convert them to integers
# y_test = np.argmax(y_test_cat, axis=1)

# 1. Overall accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f" Overall Accuracy: {accuracy:.4f}")

# 2. F1 Score (macro average)
f1_macro = f1_score(y_test, y_pred, average='macro')
print(f" Macro-averaged F1 Score: {f1_macro:.4f}")

# 3. Full classification report
target_names = le_emotion.classes_  # use label encoder to get original class names
report = classification_report(y_test, y_pred, target_names=target_names, output_dict=True)
df_report = pd.DataFrame(report).transpose()

# Show only per-class rows (filter out avg/accuracy rows if needed)
df_per_class = df_report.loc[target_names]
print("\n Per-Class Metrics (Precision, Recall, F1-score, Support):")
print(df_per_class[['precision', 'recall', 'f1-score', 'support']])


 Overall Accuracy: 0.8119
 Macro-averaged F1 Score: 0.7933

 Per-Class Metrics (Precision, Recall, F1-score, Support):
         precision    recall  f1-score  support
Angry     0.948052  0.973333  0.960526     75.0
Calm      0.840000  0.840000  0.840000     75.0
Disgust   0.814815  0.564103  0.666667     39.0
Fearful   0.765432  0.826667  0.794872     75.0
Happy     0.830986  0.786667  0.808219     75.0
Neutral   0.607843  0.815789  0.696629     38.0
Sad       0.814286  0.760000  0.786207     75.0


## **Model Training with ML Models**

In [91]:
## Intializing Models
models={"SVC":SVC(kernel='rbf'),"Logistic Regression":LogisticRegression(),"Bernouli ALgorithm":BernoulliNB(),"K Nearest Neighbor":KNeighborsClassifier(),
        "Decision Tree":DecisionTreeClassifier(),"Random Forest":RandomForestClassifier(),"Gradient Boost":GradientBoostingClassifier(),
        "Adaboost":AdaBoostClassifier(),"XGBoost":XGBClassifier()}

for i in range(len(models)):
  model=list(models.values())[i]
  model.fit(X_train_bal,y_train_bal)

  ## Making Predictions
  y_train_pred=model.predict(X_train_bal)
  y_test_pred=model.predict(X_test_scaled)

  ## Finding training set performance.
  model_train_accuracy=accuracy_score(y_train_bal,y_train_pred)
  model_train_precision=precision_score(y_train_bal,y_train_pred,average='weighted')
  model_train_recall=recall_score(y_train_bal,y_train_pred,average='weighted')
  model_train_f1=f1_score(y_train_bal,y_train_pred,average='weighted')


  ## Finding training set performance.
  model_test_accuracy=accuracy_score(y_test,y_test_pred)
  model_test_precision=precision_score(y_test,y_test_pred,average='weighted')
  model_test_recall=recall_score(y_test,y_test_pred,average='weighted')
  model_test_f1=f1_score(y_test,y_test_pred,average='weighted')


  # Printing

  print(list(models.keys())[i])  ##Printing Model Name( Stored as keys )

  print('Model performance for Training set')
  print('-Accuracy: {:.4f}'.format(model_train_accuracy))
  print('-Precision: {:.4f}'.format(model_train_precision))
  print('-Recall: {:.4f}'.format(model_train_recall))
  print('-F1 Score: {:.4f}'.format(model_train_f1))


  print("-"*50)

  print('Model performance for Test set')
  print('-Accuracy: {:.4f}'.format(model_test_accuracy))
  print('-Precision: {:.4f}'.format(model_test_precision))
  print('-Recall: {:.4f}'.format(model_test_recall))
  print('-F1 Score: {:.4f}'.format(model_test_f1))


  print("<>"*50)

SVC
Model performance for Training set
-Accuracy: 0.7499
-Precision: 0.7718
-Recall: 0.7499
-F1 Score: 0.7485
--------------------------------------------------
Model performance for Test set
-Accuracy: 0.6482
-Precision: 0.6833
-Recall: 0.6482
-F1 Score: 0.6512
<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
Logistic Regression
Model performance for Training set
-Accuracy: 0.7646
-Precision: 0.7640
-Recall: 0.7646
-F1 Score: 0.7634
--------------------------------------------------
Model performance for Test set
-Accuracy: 0.6571
-Precision: 0.6610
-Recall: 0.6571
-F1 Score: 0.6568
<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
Bernouli ALgorithm
Model performance for Training set
-Accuracy: 0.3351
-Precision: 0.3506
-Recall: 0.3351
-F1 Score: 0.2851
--------------------------------------------------
Model performance for Test set
-Accuracy: 0.3363
-Precision: 0.3549
-Recall: 0.3

Not upto the mark result we got in ANN models

## Tuning best ML models

In [92]:
##Defining parameter grid for our different models.
rf_params={"max_depth":[8,10,15,17,None],
           "max_features":[5,7,"auto",8],
            "min_samples_split": [2, 8, 15, 20],
            "n_estimators": [100, 200, 500, 1000],
           "criterion":['gini','entropy','log_loss']}
gb_params={"learning_rate":[0.1,0.01,0.2],
           "n_estimators":[100,200,300,400],
           "criterion":['friedman_mse','squared_error'],
           "min_samples_split":[2,4,6]}
xgb_params={"learning_rate":[0.01,0.1,0.2],
            "max_depth":[5,8,12,20,30],
            "n_estimator":[100,200,300,400],
            "colsample_bytree":[0.5,0.8,1,0.3,0.4]}

In [93]:
##Model list for hyper parameter tuning.
randomcv_models = [("XGB",XGBClassifier(),xgb_params)]

In [94]:
from sklearn.model_selection import RandomizedSearchCV
model_param={}
for name,model,params in randomcv_models:
  random=RandomizedSearchCV(estimator=model,
                            param_distributions=params,
                            cv=3,
                            verbose=2,
                            n_jobs=-1)
  random.fit(X_train,y_train)
  model_param[name]=random.best_params_

for model_name in model_param:
  print(f"---------------- Best Params for {model_name} -------------------")
  print(model_param[model_name])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
---------------- Best Params for XGB -------------------
{'n_estimator': 100, 'max_depth': 12, 'learning_rate': 0.2, 'colsample_bytree': 0.3}


In [96]:
models={

    "Random Forest":RandomForestClassifier(n_estimators= 200, min_samples_split= 2,max_features= 5, max_depth= None, criterion= 'log_loss'),
    "Gradient Boost":GradientBoostingClassifier(n_estimators=400, min_samples_split= 2, learning_rate= 0.2,criterion= 'squared_error'),
    "XG Boost":XGBClassifier(n_estimator= 100, max_depth= 12, learning_rate= 0.2,colsample_bytree= 0.3)
}



for i in range(len(models)):
  model=list(models.values())[i]
  model.fit(X_train,y_train)

  ## Making Predictions
  y_train_pred=model.predict(X_train)
  y_test_pred=model.predict(X_test)

  ## Finding training set performance.
  model_train_accuracy=accuracy_score(y_train,y_train_pred)
  model_train_precision=precision_score(y_train,y_train_pred,average='weighted')
  model_train_recall=recall_score(y_train,y_train_pred,average='weighted')
  model_train_f1=f1_score(y_train,y_train_pred,average='weighted')


  ## Finding training set performance.
  model_test_accuracy=accuracy_score(y_test,y_test_pred)
  model_test_precision=precision_score(y_test,y_test_pred,average='weighted')
  model_test_recall=recall_score(y_test,y_test_pred,average='weighted')
  model_test_f1=f1_score(y_test,y_test_pred,average='weighted')


  # Printing

  print(list(models.keys())[i])  ##Printing Model Name( Stored as keys )

  print('Model performance for Training set')
  print('-Accuracy: {:.4f}'.format(model_train_accuracy))
  print('-Precision: {:.4f}'.format(model_train_precision))
  print('-Recall: {:.4f}'.format(model_train_recall))
  print('-F1 Score: {:.4f}'.format(model_train_f1))


  print("-"*50)

  print('Model performance for Test set')
  print('-Accuracy: {:.4f}'.format(model_test_accuracy))
  print('-Precision: {:.4f}'.format(model_test_precision))
  print('-Recall: {:.4f}'.format(model_test_recall))
  print('-F1 Score: {:.4f}'.format(model_test_f1))


  print("<>"*50)

Random Forest
Model performance for Training set
-Accuracy: 1.0000
-Precision: 1.0000
-Recall: 1.0000
-F1 Score: 1.0000
--------------------------------------------------
Model performance for Test set
-Accuracy: 0.6770
-Precision: 0.6843
-Recall: 0.6770
-F1 Score: 0.6732
<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
Gradient Boost
Model performance for Training set
-Accuracy: 1.0000
-Precision: 1.0000
-Recall: 1.0000
-F1 Score: 1.0000
--------------------------------------------------
Model performance for Test set
-Accuracy: 0.7699
-Precision: 0.7691
-Recall: 0.7699
-F1 Score: 0.7687
<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
XG Boost
Model performance for Training set
-Accuracy: 1.0000
-Precision: 1.0000
-Recall: 1.0000
-F1 Score: 1.0000
--------------------------------------------------
Model performance for Test set
-Accuracy: 0.7500
-Precision: 0.7511
-Recall: 0.7500
-